In [ ]:
import opinion_clob_sdk

# Check version
print(opinion_clob_sdk.__version__)  # Should print: 0.1.0 or higher

# Import main classes
from opinion_clob_sdk import Client
from opinion_clob_sdk.model import TopicType, TopicStatus

print("✓ Opinion CLOB SDK installed successfully!")

In [ ]:
import requests

BASE = "https://api.opinion.trade"

def get_all_markets(page=1, limit=20):
    url = f"{BASE}/openapi/market?page=page&limit={limit}"
    return requests.get(url)



In [ ]:
get_all_markets(page=1, limit=20)

In [ ]:
import requests

BASE = "https://api.opinion.trade"

def get_markets(topic_id):
    url = f"{BASE}/markets?topicId={topic_id}"
    return requests.get(url).json()

def get_orderbook(market_id):
    url = f"{BASE}/orderbooks?marketId={market_id}"
    return requests.get(url).json()

topic_id = 80

markets = get_markets(topic_id)
print(markets)


In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def sniff_all_requests(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        print("Listening for all network requests...\n")

        page.on("request", lambda req: print("REQ →", req.url))
        page.on("response", lambda res: print("RES ←", res.url))

        await page.goto(url)
        await page.wait_for_timeout(5000)

        await browser.close()


url = "https://app.opinion.trade/detail?topicId=1274"
await sniff_all_requests(url)


In [ ]:
import asyncio
import json
from playwright.async_api import async_playwright

async def get_opinion_orderbook(topic_id):
    url = f"https://app.opinion.trade/detail?topicId={topic_id}"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        symbols = {}
        question_id = None

        def handle_request(req):
            u = req.url

            if "order/market/depth" in u:
                # symbol info will appear here
                print("FOUND DEPTH REQ →", u)

                params = dict([part.split("=") for part in u.split("?")[1].split("&")])

                if "symbol_types" in params and "symbol" in params:
                    symbols[int(params["symbol_types"])] = params["symbol"]

                nonlocal question_id
                if "question_id" in params:
                    question_id = params["question_id"]

        page.on("request", handle_request)

        await page.goto(url)
        await page.wait_for_timeout(5000)

        await browser.close()

        return symbols, question_id


async def fetch_depth(symbol, question_id, symbol_type):
    import httpx
    api = "https://proxy.opinion.trade:8443/api/bsc/api/v2/order/market/depth"

    params = {
        "symbol": symbol,
        "symbol_types": symbol_type,
        "question_id": question_id,
        "chainId": 56
    }

    async with httpx.AsyncClient() as client:
        r = await client.get(api, params=params)
        return r.json()


async def main():
    symbols, qid = await get_opinion_orderbook(1274)
    print("Symbols:", symbols)
    print("Question ID:", qid)

    yes_depth = await fetch_depth(symbols[0], qid, 0)
    no_depth = await fetch_depth(symbols[1], qid, 1)

    print("\nYES depth:")
    print(yes_depth)

    print("\nNO depth:")
    print(no_depth)


await main()


In [ ]:
import httpx
import asyncio

BASE = "https://proxy.opinion.trade:8443/api/bsc/api/v2"

async def get_topic_info(topic_id):
    url = f"{BASE}/topic/{topic_id}"
    async with httpx.AsyncClient() as client:
        r = await client.get(url)
        data = r.json()
        return data["result"]


async def get_depth(symbol, qid, symbol_type):
    url = f"{BASE}/order/market/depth"
    params = {
        "symbol": symbol,
        "symbol_types": symbol_type,
        "question_id": qid,
        "chainId": 56,
    }
    async with httpx.AsyncClient() as client:
        r = await client.get(url, params=params)
        return r.json()["result"]


async def get_orderbook(topic_id):
    # 0.15s
    info = await get_topic_info(topic_id)

    # 从 topic API 中直接拿 YES/NO symbol 和 qid
    yes_symbol  = info["yesToken"]
    no_symbol   = info["noToken"]
    qid         = info["questionId"]

    # 两个 depth 接口并行 (0.1s)
    yes_task = get_depth(yes_symbol, qid, 0)
    no_task  = get_depth(no_symbol, qid, 1)

    yes_depth, no_depth = await asyncio.gather(yes_task, no_task)

    return {"YES": yes_depth, "NO": no_depth}


# ==== 测试 ====
async def main():
    data = await get_orderbook(1274)
    print(data)

await main()



In [ ]:
info = await get_topic_info(1274)
print(info)

In [24]:
import os
from dotenv import load_dotenv
from opinion_clob_sdk import Client

# Load environment variables
load_dotenv()

# Initialize client
client = Client(
    host='https://proxy.opinion.trade:8443',
    apikey=os.getenv('API_KEY'),
    chain_id=56,  # BNB Chain mainnet
    rpc_url=os.getenv('RPC_URL'),
    private_key=os.getenv('PRIVATE_KEY'),
    multi_sig_addr=os.getenv('MULTI_SIG_ADDRESS'),
    conditional_tokens_addr=os.getenv('CONDITIONAL_TOKEN_ADDR'),
    multisend_addr=os.getenv('0x998739BFdAAdde7C933B942a68053933098f9EDa')
)

print("✓ Client initialized successfully!")

✓ Client initialized successfully!


In [19]:
from opinion_clob_sdk.model import TopicStatusFilter

# Get all active markets
markets_response = client.get_markets(
    status=TopicStatusFilter.ACTIVATED,
    page=3,
    limit=10
)

In [20]:
markets_response

OpenapiMarketGet200Response(errmsg='', errno=0, result=OpenapiMarketListRespOpenAPI(list=[OpenapiMarketDataOpenAPI(chain_id='56', child_markets=None, condition_id='', created_at=1762327856, cutoff_at=1798675200, market_id=1546, market_title="Supreme Court rules in favor of Trump's tariffs?", no_label='NO', no_token_id='80993736742659296074271471707092779338813385409282660753587012054032463729137', question_id='6b55368b7b9ce622a203943c17ee477a8f84be21ded8313efc8d8ee1d7be56bb', quote_token='0x55d398326f99059fF775485246999027B3197955', resolved_at=0, result_token_id='', rules='This market will resolve to "Yes" if the Supreme Court of the United States issues a decision that reverses, vacates, or otherwise overturns the U.S. Court of Appeals for the Federal Circuit’s August 29, 2025 decision in V.O.S. Selections, Inc. v. Trump, in which the Federal Circuit held that the tariffs imposed by Executive Orders 14193, 14194, 14195, 14257, and 14266 exceeded the President’s authority under the In

In [21]:
markets = markets_response.result.list

print(f"Found {len(markets)} active markets:\n")

for m in markets:
    print(
        f"- ID={m.market_id} | "
        f"Title={m.market_title} | "
        f"Status={m.status_enum} | "
        f"Volume={m.volume}"
    )


Found 8 active markets:

- ID=1546 | Title=Supreme Court rules in favor of Trump's tariffs? | Status=Activated | Volume=595187.409000000000000000
- ID=1463 | Title=USDT depeg in 2025? | Status=Activated | Volume=235845403.002000000000000000
- ID=1384 | Title=Another crypto hack over $100m in 2025 (After Nov 4)? | Status=Activated | Volume=1249268.627000000000000000
- ID=1377 | Title=Hyperliquid all time high by Dec 31? | Status=Activated | Volume=1407053.028000000000000000
- ID=1308 | Title=Kraken IPO in 2025? | Status=Activated | Volume=19550721.240000000000000000
- ID=1284 | Title=TikTok sale announced by Dec 31? | Status=Activated | Volume=916318.915000000000000000
- ID=1278 | Title=Russia x Ukraine ceasefire in 2025? | Status=Activated | Volume=8243300.855000000000000000
- ID=1277 | Title=Ethereum all time high by Dec 31? | Status=Activated | Volume=4705251.882000000000000000


In [25]:
from opinion_clob_sdk import Client
from opinion_clob_sdk.model import TopicStatusFilter


# 获取所有激活市场
resp = client.get_markets(
    status=TopicStatusFilter.ACTIVATED,
    page=1,
    limit=50
)

markets = resp.result.list

for m in markets:
    print("====================================")
    print(f"Market ID: {m.market_id}")
    print(f"Title:     {m.market_title}")
    print(f"YES label: {m.yes_label}")
    print(f"YES token: {m.yes_token_id}")
    print(f"NO label:  {m.no_label}")
    print(f"NO token:  {m.no_token_id}")


InvalidParamError: limit must be between 1 and 20

In [26]:
from opinion_clob_sdk.model import TopicStatusFilter

# Get all active markets
markets_response = client.get_markets(
    status=TopicStatusFilter.ACTIVATED,
    page=1,
    limit=10
)
markets = markets_response.result.list
# Parse the response
if markets_response.errno == 0:
    markets = markets_response.result.list
    print(f"\n✓ Found {len(markets)} active markets:")

    for market in markets[:3]:  # Show first 3
        #print(f"  - Market #{market.marketId}: {market.marketTitle}")
        print(f"    Status: {market.status}")
        print()
else:
    print(f"Error: {markets_response.errmsg}")


✓ Found 3 active markets:
    Status: ModelsTopicStatus.TopicStatusActivated

    Status: ModelsTopicStatus.TopicStatusActivated

    Status: ModelsTopicStatus.TopicStatusActivated



In [28]:
markets

[OpenapiMarketDataOpenAPI(chain_id='56', child_markets=None, condition_id='', created_at=1764253337, cutoff_at=1764374400, market_id=1990, market_title='Bitcoin Up or Down on November 28?(By 12:00 ET)', no_label='DOWN', no_token_id='97885733047754836919169601360615688894990645400941582245393933892136731330999', question_id='92699559e7ee13c0390f4b6859f5f143dd3ff9a317a34ce2259d298eb1fdd8be', quote_token='0x55d398326f99059fF775485246999027B3197955', resolved_at=0, result_token_id='', rules='This market will resolve to "Up" if the "Close" price for the Binance 1 minute candle for BTC/USDT Nov 27 \'25 12:00 in the ET timezone (noon) is lower than the final "Close" price for the Nov 28 \'25 12:00 ET candle.\n\nThis market will resolve to "Down" if the "Close" price for the Binance 1 minute candle for BTC/USDT Nov 27 \'25 12:00 in the ET timezone (noon) is higher than the final "Close" price for the Nov 28 \'25 12:00 ET candle.\n\nIf the final "Close" price for both of these candles is exactl

In [29]:
# Assuming the market has a token (get from market.options for binary markets)
# For this example, we'll use a placeholder token_id
token_id = "108018987032839108083616928409013432500383448394358283321664894825987333118778"  # Replace with actual token ID

try:
    orderbook = client.get_orderbook(token_id)
    if orderbook.errno == 0:
        book = orderbook.result.data
        print(f"\n✓ Orderbook for token {token_id}:")
        print(f"  Best Bid: {book.bids[0] if book.bids else 'No bids'}")
        print(f"  Best Ask: {book.asks[0] if book.asks else 'No asks'}")
except Exception as e:
    print(f"  (Skip if token_id not set: {e})")

  (Skip if token_id not set: 'OpenapiOrderbookRespOpenAPI' object has no attribute 'data')


In [30]:
orderbook

OpenapiTokenOrderbookGet200Response(errmsg='', errno=0, result=OpenapiOrderbookRespOpenAPI(asks=[OpenapiOrderbookLevel(price='0.999', size='205394.89'), OpenapiOrderbookLevel(price='0.98', size='16136.884081632654'), OpenapiOrderbookLevel(price='0.985', size='2278.63'), OpenapiOrderbookLevel(price='0.988', size='1000000'), OpenapiOrderbookLevel(price='0.99', size='31.36'), OpenapiOrderbookLevel(price='0.998', size='42318.8'), OpenapiOrderbookLevel(price='0.981', size='4522.31'), OpenapiOrderbookLevel(price='0.984', size='964.44'), OpenapiOrderbookLevel(price='0.986', size='3159.67'), OpenapiOrderbookLevel(price='0.996', size='2500'), OpenapiOrderbookLevel(price='0.978', size='72.44272727272727'), OpenapiOrderbookLevel(price='0.979', size='50428.42926455567'), OpenapiOrderbookLevel(price='0.982', size='56046.950000000004'), OpenapiOrderbookLevel(price='0.997', size='33333.33')], bids=[OpenapiOrderbookLevel(price='0.974', size='403.12'), OpenapiOrderbookLevel(price='0.973', size='411.08'